# A Star Graph Search algorithm

In [2]:
#-------------leetcode 1091-----------------
"""shortest path in binary matrix: n*n squared grid, each cell is either empty or blicked (1) 
a clear path from grid[0][0] to grid[n-1][n-1], through empty cells floow the 8directions. return
-1 if no such path."""
# A* Algorithm
class Solution:
    def shortestPathBinaryMatrix(self, grid):
        if not grid or len(grid)==0 or len(grid[0])==0:
            return -1
        shortest_path = a_star_graph_search(start=(0,0),goal_function=get_goal_function(grid),
                                           successor_function=get_successor_function(grid),
                                           heuristic=get_heuristic(grid))
        if shortest_path is None or grid[0][0]==1:
            return -1
        else:
            return len(shortest_path)
        
    def a_star_graph_search(start, goal_function,successor_function,heuristic):
        visited = set()
        came_from = dict()
        distance = {start:0}
        frontier = PriorityQueue()
        frontier.add(start)
        while frontier:
            node = frontier.pop()
            if node in visited:
                continue
            if goal_function(node):
                return reconstruct_path(came_from, start, node)
            visited.add(node)
            for successor in successor_function(node):
                frontier.add(successor, priority=distance[node]+1+heuristic(successor))
                if successor not in distance or distance[node]+1 <distance[successor]:
                    distance[successor] = distance[node]+1
                    came_from[successor] = node
        return None
    
    def reconstruct_path(came_from, start, end):
        """
        >>>came_from = {'b':'a','c':'a','d':'c','e':'d','f':'d'}
        >>>reconstruct_path(came_from, 'a','e')
        ['a','c','d','e']
        """
        reverse_path = [end]
        while end != start:
            end = came_from[end]
            reverse_path.append(end)
        return list(reverse(reverse_path))
    
    # Goal function---a function to check whether we have reached the goal cell
    def get_goal_function(grid):
        """
        >>> f = get_goal_function([[0,0],[0,0]])
        >>> f((0,0))
        False
        >>> f((0,1))
        False
        >>> f((1,1))
        True
        """
        M, N = len(grid), len(grid[0])
        def is_bottom_right(cell):
            return cell == (M-1, N-1)
        return is_bottom_right
    
    # Successor function--- a function to find the cells adjancent to the current cell
    def get_successor_function(grid):
        """
        >>> f = get_successor_function([[0,0,0],[0,1,0],[1,0,0]])
        >>> sorted(f((1,2)))
        [(0,1),(0,2),(2,1),(2,2)]          0 0 0
        >>> sorted(f((2,1)))               0 1 0 
        [(1,0),(1,2),(2,2)]                1 0 0
        """
        # find the 8 directions adjacent 0s
        def get_clear_adjacent_cells(cell):
            i,j =cell
            return ((i+a, j+b) for a in (-1,0,1) for b in (-1,0,1) 
                   if a != 0 or b != 0 
                   if 0 <= i+a < len(grid)
                   if 0 <= j+b < len(grid[0])
                   if grid[j+a][j+b] ==0
                   )
        return get_clear_adjacent_cells
    
    # Heuristic function---a function to find the cells adjacent to the current cell
    def get_heuristic(grid):
        """
        >>> f = get_heuristic([[0,0],[0,0]])
        >>> f((0,0))
        1
        >>> f((0,1))
        1
        >>> f((1,1))
        0
        """
        M,N = len(grid), len(grid[0])
        (a,b) = goal_cell = (M-1, N-1)
        def get_clear_path_distance_from_goal(cell):
            (i,j) = cell
            return max(abs(a-i), abs(b-j))
        return get_clear_path_distance_from_goal
    
    
# Priority Queue Class (need self construct)
from heapq import heappush, heappop
class PriorityQueue:
    def __init__(self, iterable=[]):
        self.heap =[]
        for value in iterable:
            heappush(self.heap, (0,value))
            
    def add(self, value, priority=0):
        heappush(self.heap, (priority, value))
        
    def pop(self):
        priority, value = heappop(self.heap)
        return value
    
    def __len__(self):
        return len(self.heap)

  